In [17]:
# Imports
import os
import pandas as pd
import numpy as np

In [18]:
# Identify directory for U-Pb data
directory_no_filter = os.path.normpath('../upb_data_agecalc/no_filter/')
directory_rel_age_filter = os.path.normpath('../upb_data_agecalc/rel_age_filter/')

In [19]:
# Make a class for AgeCalc DZ data
class AgeCalcDZ:
    def __init__(self,name,accepted,rejected,standards,syst_238,syst_207):

        self.name = name
        self.accepted = accepted
        self.rejected = rejected
        self.standards = standards
        self.syst_238 = syst_238
        self.syst_207 = syst_207

        return

In [20]:
# Define function to process directory

def proc_directory(directory):
    print('Source Directory: ',directory)

    output_dir = 'proc_'+ os.path.basename(directory)

    os.makedirs(output_dir,exist_ok=True)
    print('Output Directory: ',output_dir)

    new_columns = (
    ['Analysis','U','206Pb/204Pb','U/Th','206Pb/207Pb','206Pb/207Pb_err','207Pb/235U','207Pb/235U_err','206Pb/238U','206Pb/238U_err','error_corr','206Pb/238U_age','206Pb/238U_ageerr','207Pb/235U_age','207Pb/235U_ageerr','206Pb/207Pb_age','206Pb/207Pb_ageerr','Best age','Best age_err','Conc']
    )

    for root,dirs,files in os.walk(directory):
        # Get names from dirs
        if len(dirs)>1:
            names = dirs
            print(names)
        
        for file in files:
            if ('_DataTable.xls' in file) & ('lock' not in file):
                path = os.path.join(root,file)
                
                # Read relevant part of the Excel file
                df = pd.read_excel(path,skiprows=25,usecols='A:T',header=None,sheet_name='Sheet1')
                df.columns = np.arange(20)

                reject = pd.read_excel(path,skiprows=25,usecols='W:AP',header=None,sheet_name='Sheet1')
                reject.columns = np.arange(20)

                stand = pd.read_excel(path,skiprows=25,usecols='AS:BL',header=None,sheet_name='Sheet1')
                stand.columns = np.arange(20)

                syst = pd.read_excel(path,skiprows=9,nrows=2,header=None,
                                            usecols='B')

                syst_238 = syst.loc[0,1]
                syst_207 = syst.loc[1,1]
                
                # Remove empty rows
                df.dropna(how='all',inplace=True)
                reject.dropna(how='all',inplace=True)
                
                # Move SLs to standards
                sl_bool = df.iloc[:,0].str.contains('SL')

                sl = df[sl_bool==True]
                df = df[sl_bool==False]

                # Remove rejected standards
                reject.iloc[:,0] = reject.iloc[:,0].astype(str)

                std_reject = (
                    (reject.iloc[:,0].str.contains('SL'))|
                    (reject.iloc[:,0].str.contains('F'))|
                    (reject.iloc[:,0].str.contains('R'))
                )

                reject = reject[std_reject==False]

                name = file.partition('_')[0]

                sample = AgeCalcDZ(name=name,accepted=df,rejected=reject,standards=stand,syst_238=syst_238,syst_207=syst_207)
                print(sample.name, 'n=',len(sample.accepted))

                output = sample.accepted
                output.columns = new_columns

                output.to_csv(output_dir + '/' + sample.name + '.csv')

In [22]:
# Process no filter

proc_directory(directory_no_filter)


Source Directory:  ..\upb_data_agecalc\no_filter
Output Directory:  proc_no_filter
100211-1A n= 105
100211-9A n= 302
100411-5 n= 41
AB0913 n= 179
AB0926 n= 213
CT130918-2A n= 310
CT130918-9A n= 292
CT130919-5A n= 296
CT130919-8A n= 10
CT15076 n= 290
CT15082 n= 10
CT15092 n= 185
CT15099 n= 10
CT15113 n= 81
CT15127 n= 5


In [23]:
# Prcoess relative age filter
proc_directory(directory_rel_age_filter)

Source Directory:  ..\upb_data_agecalc\rel_age_filter
Output Directory:  proc_rel_age_filter
100211-1A n= 99
100211-9A n= 295
100411-5 n= 39
AB0913 n= 179
AB0926 n= 197
CT130918-2A n= 285
CT130918-9A n= 286
CT130919-5A n= 290
CT130919-8A n= 10
CT15076 n= 283
CT15082 n= 10
CT15092 n= 183
CT15099 n= 10
CT15113 n= 79
CT15127 n= 5
